# How to build a simple Chatbot with stored memory using LangChain
* Simple Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to store memory in a json file.


A chatbot app is a software application designed to simulate conversation with human users. It uses artificial intelligence (AI) to understand what someone is saying and to respond with appropriate answers. These apps can be used for various purposes like customer support, providing information, or entertainment. Essentially, it's like texting with a program that can answer questions and help with tasks.

## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.

## Trick to avoid the nasty deprecation warnings from LangChain

In this exercise we will use the LangChain legacy chain LLMChain. It works well, but LangChain displays a nasty deprecation warning. To avoid it, we will enter the following code:

In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [ ]:
#!pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [5]:
#!pip install langchain-openai

* For this project, we will use Groqs's llama-3.1-8b-instant

In [4]:
from langchain_groq import ChatGroq

chatbot = ChatGroq(model = "llama-3.1-8b-instant")

* Human Message: the user input.

In [5]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue."),
]

#### Call the ChatModel (the LLM)

In [6]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Blue is a beautiful and calming color. It can evoke feelings of serenity, trust, and stability. Many people find blue to be a soothing and relaxing color, which is why it's often associated with the sky and the ocean.\n\nThere are also many different shades of blue, each with its own unique character. Do you have a specific shade of blue that you like best, such as navy, royal blue, or sky blue?", response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 41, 'total_tokens': 129, 'completion_time': 0.114893197, 'prompt_time': 0.002422617, 'queue_time': 0.054741163, 'total_time': 0.117315814}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'finish_reason': 'stop', 'logprobs': None}, id='run-096239d4-3a86-4d22-b59e-74aef8b603b7-0', usage_metadata={'input_tokens': 41, 'output_tokens': 88, 'total_tokens': 129})

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Check if the Chatbot remembers your favorite color.

In [7]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm not aware of any information about your favorite color. I'm a large language model, I don't have personal knowledge about individuals, and our conversation just started. If you'd like to share your favorite color, I'd be happy to chat with you about it!", response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 41, 'total_tokens': 97, 'completion_time': 0.063440976, 'prompt_time': 0.003699548, 'queue_time': 0.050189662, 'total_time': 0.067140524}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'finish_reason': 'stop', 'logprobs': None}, id='run-45ef18db-85b5-4394-a207-c3e7fa3f43fb-0', usage_metadata={'input_tokens': 41, 'output_tokens': 56, 'total_tokens': 97})

* As you can see, our Chatbot cannot remember our previous interaction.

## Let's add memory to our Chatbot

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [10]:
#!pip install langchain_community

In [8]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [9]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [10]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [11]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

In [12]:
chain.invoke("hello!")

{'content': 'hello!',
 'messages': [],
 'text': 'Hello! How can I assist you today?'}

In [13]:
chain.invoke("my name is mursalin")

{'content': 'my name is mursalin',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': 'Nice to meet you, Mursalin. Is there something I can help you with or would you like to chat?'}

In [14]:
chain.invoke("what is my name?")

{'content': 'what is my name?',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='my name is mursalin'),
  AIMessage(content='Nice to meet you, Mursalin. Is there something I can help you with or would you like to chat?')],
 'text': 'Your name is Mursalin.'}

* Check the file messages.json in the root directory.
* This is just a simple example, in the real world you probably will not save your memory in a json file. 
* And remember: the context window is limited and it affects to the cost of using  Chat Models API.